## !Important to do this first to have the effnetv2 model installed!

In [1]:
%%capture
#@title
!pip install tensorflow_addons

import os
import sys
import tensorflow.compat.v1 as tf

# Download source code.
if "efficientnetv2" not in os.getcwd():
    !git clone --depth 1 https://github.com/google/automl
    os.chdir('automl/efficientnetv2')
    sys.path.append('.')
else:
    !git pull

def download(m):
    if m not in os.listdir():
        !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/v2/{m}.tgz
        !tar zxf {m}.tgz
    ckpt_path = os.path.join(os.getcwd(), m)
    return ckpt_path

## Import my funcs and other necessary libraries

In [2]:
import EfficientNet_Functions as myfuncs

In [3]:
import tensorflow_hub as hub
import itertools
import os

import matplotlib.pylab as plt
import numpy as np
import csv
import PIL
import PIL.Image
import tensorflow as tf
#import tensorflow_datasets as tfds
import pathlib
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

## Configs

In [4]:
data_dir_train = '/projectnb/dl523/students/nannkat/EC520/wikipaintings_full/content/gdrive/Shareddrives/520 Project/Data/wikipaintings_full_aug/train'
data_dir_val = '/projectnb/dl523/students/nannkat/EC520/wikipaintings_full/wikipaintings_val'
data_dir_test = '/projectnb/dl523/students/nannkat/EC520/wikipaintings_full/wikipaintings_test'

checkpoint_path = "/projectnb/dl523/students/nannkat/EC520/training/cp.ckpt"
model_path = '/projectnb/dl523/students/nannkat/EC520/training/effnetv2_model'

IMAGE_SIZE = 223
BATCH_SIZE =  64 
LEARNING_RATE = 0.001

## Load data

In [ ]:
_,_,test_generator = myfuncs.load_wikidata(data_dir_train, data_dir_val, data_dir_test,
                                                                batch_size = BATCH_SIZE, image_size = IMAGE_SIZE)

myfuncs.visualize_input(test_generator)

## Load trained model

In [6]:
initialize = False

if initialize:
    model = myfuncs.get_new_model()
else:
    model = myfuncs.restore_model(checkpoint_path, learning_rate = LEARNING_RATE)

Instructions for updating:
Restoring a name-based tf.train.Saver checkpoint using the object-based restore API. This mode uses global names to match variables, and so is somewhat fragile. It also adds new restore ops to the graph each time it is called when graph building. Prefer re-encoding training checkpoints in the object-based format: run save() on the object-based saver (the same one this message is coming from) and use that checkpoint in the future.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-l (EffNetV2M  (None, 1280)             117746848 
 odel)                                                           
                                                                 
 dropout_1 (Dropout)         (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 25)                32025     
                                                                 
Total params: 117,778,873
Trainable params: 32,025
Non-trainable params: 117,746,848
_________________________________________________________________


## Evaluate

In [7]:
test_steps = test_generator.samples // test_generator.batch_size
model.evaluate(test_generator, steps = test_steps)

128/128 [==============================] - 305s 2s/step - loss: 2.0470 - accuracy: 0.4508 - top_k_categorical_accuracy: 0.8481


[2.046984910964966, 0.4508056640625, 0.84814453125]

## Example predictions

In [ ]:
myfuncs.visualize_results(model, test_generator)

## Predict and save .csv

In [10]:
output_path = '/projectnb/dl523/students/nannkat/EC520/preds_vs_true_effnet_1Freeze10epochLr001.csv'
preds, true_labels, preds_df = myfuncs.predict_and_save(model, test_generator, output_path)

Predicting....
128/129 [============================>.] - ETA: 16s